In [216]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [194]:
df = pd.read_csv("diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [208]:
threshold = 0.5
x = df[["Age", "DiabetesPedigreeFunction"]].values
y = df["Outcome"]
true_y = df["Outcome"]
f"Shapes of x = {x.shape} y={y.shape}, #Outcome = {y.sum()}"

'Shapes of x = (768, 2) y=(768,), #Outcome = 268'

In [196]:
X = StandardScaler().fit_transform(x)
pipe = Pipeline([
    ("scale", StandardScaler()),
    ("model", KNeighborsRegressor(n_neighbors=3))
])

In [197]:
mod = GridSearchCV(estimator = pipe,
                  param_grid = {'model__n_neighbors':[3]},
                   cv = 3)
mod.fit(X, y)
pd.DataFrame(mod.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004155,0.00098,0.002666,0.000475,3,{'model__n_neighbors': 3},-0.15365,-0.321333,-0.196493,-0.223825,0.071132,1


In [210]:
knn = (mod.predict(X) > threshold).astype(int)
knn.sum()

256

In [199]:
lreg = LogisticRegression(class_weight = {0:1, 1:1.5}, max_iter = 1000)
lreg.fit(X,y).predict(X).sum()

224

In [200]:
grid = GridSearchCV(estimator = lreg,
                   param_grid = {'class_weight': [{0:1, 1:4}]},
                   cv = 3,
                   n_jobs = -1)
grid.fit(x, y)
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011001,0.001631,0.001334,0.000471,"{0: 1, 1: 4}","{'class_weight': {0: 1, 1: 4}}",0.46875,0.523438,0.414062,0.46875,0.044652,1


In [212]:
lin = (grid.predict(X) > threshold).astype(int)
lin.sum()

67

In [202]:
dtree = DecisionTreeClassifier()
dtree.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [203]:
tree = GridSearchCV(estimator = dtree,
                    param_grid = {'class_weight': [{0:1, 1:1}]},
                   cv = 3)
tree.fit(X, y)
pd.DataFrame(tree.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.00529,0.001139,0.00169,0.000926,"{0: 1, 1: 1}","{'class_weight': {0: 1, 1: 1}}",0.621094,0.605469,0.605469,0.610677,0.007366,1


In [213]:
dec = (tree.predict(X) > threshold).astype(int)
dec.sum()

264

In [218]:
print(accuracy_score(y, knn))
print(accuracy_score(y, lin))
print(accuracy_score(y, dec))

0.7890625
0.66015625
0.9947916666666666
